In [171]:
import torch
from torch import tensor
from typing import List, Optional

# MyScalar class

In [172]:
class MyScalar:
	def __init__(self, value, gradient = 1, parent = None):
		self.value = value
		self.gradient = gradient
		self.parent = parent

	def __repr__(self):
		return f"MyScalar({self.value}, {self.gradient}, {self.parent})"

# Function Library

In [173]:
def my_add(scalar: MyScalar, n):
    value = scalar.value + n
    grad = 1
    return MyScalar(value, grad, scalar)

def my_mul(scalar: MyScalar, n):
    value = n * scalar.value
    grad = n
    return MyScalar(value, grad, scalar)

def my_pow(scalar: MyScalar, n):
    value = scalar.value ** n
    grad = n * (scalar.value ** (n-1))
    return MyScalar(value, grad, scalar)

def my_cos(scalar: MyScalar):
    value = torch.cos(scalar.value)
    grad = -torch.sin(scalar.value)
    return MyScalar(value, grad, scalar)

def my_sin(scalar: MyScalar):
    value = torch.sin(scalar.value)
    grad = torch.cos(scalar.value)
    return MyScalar(value, grad, scalar)

def my_ln(scalar: MyScalar):
    value = torch.log(scalar.value)
    grad = 1/(scalar.value)
    return MyScalar(value, grad, scalar)

def my_exp(scalar: MyScalar):
	# e^a when 'a' is scalar, is 1*e^a = e^a
    value = torch.exp(torch.tensor(scalar.value))
    grad = torch.exp(torch.tensor(scalar.value))
    return MyScalar(value, grad, scalar)


# get_gradient

In [174]:
global grad_graph
grad_graph = {}

def get_gradient(a: MyScalar, cumgrad=1):
	if a.parent is None:
		grad_graph[a] = cumgrad
	else:
		grad_graph[a] = cumgrad
		get_gradient(a.parent, a.gradient * cumgrad)
	return grad_graph

# Tests

In [178]:
grad_graph = {}  # Like .grad = None
my_x = MyScalar(5) # x = 5 -> value = 5, gradient = 1, parent=None
# Gradient = 1 because (df/dx)(x) = 1
print(get_gradient(my_x))

x = torch.tensor([5.], requires_grad=True)
x.backward()
print(x.grad, x.item())

{MyScalar(5, 1, None): 1}
tensor([1.]) 5.0


In [179]:
grad_graph = {}
my_x = MyScalar(5) # x = 5, value = 5, gradient = 1, parent=None
my_y = my_mul(my_x, 3) # y = 3x , value = 3*5 = 15, gradient = 3, parent=my_x
print(get_gradient(my_y))

{MyScalar(15, 3, MyScalar(5, 1, None)): 1, MyScalar(5, 1, None): 3}


`z = e^(a^2)`


`dz/da = dz*(a^2)/da * dz*e^(a^2)`


`dz/da = 2a * e^(a^2) = 4 * 54.598 = 218.4`

In [177]:
grad_graph = {} 
a=MyScalar(2) # a = 2, value = 2, gradient = 1, parent=None
b=my_pow(a,2) # b=a^2, value = 4, gradient = 2a = 4, parent=a
c=my_exp(b) # c=e^(a^2) = e^4 = 54.59815..., value = 54.5918, gradient = 54.5918, parent=b
d=get_gradient(c)
print(d)

torch_a = torch.tensor([2.], requires_grad=True)
torch_b = torch.pow(torch_a, 2)
torch_c = torch.exp(torch_b)
torch_c.backward()
print("\n\n", torch_a.grad)

{MyScalar(54.59815216064453, 54.59815216064453, MyScalar(4, 4, MyScalar(2, 1, None))): 1, MyScalar(4, 4, MyScalar(2, 1, None)): tensor(54.5982), MyScalar(2, 1, None): tensor(218.3926)}


 tensor([218.3926])
